The objective of this side-project is to investigate how we can use simulation to gain insight on the strategy behind Blackjack.

Start by building a very simple model - and build up from there. Use low-tech methods, and progressively build, following the calling of the questions you ask. This could be a very interesting to develop your understanding of reinforcement learning.

Read as little as possible. Follow the question you are asking for now. 

Objective 1a. - Build a working model of one round of Blackjack
Objective 1b. - Build a working model of multiple rounds of Blackjack

Parameters

* No. of decks - 1
* Two players - dealer and player
* Rules for the dealer
* Rules for the player
* No betting
* Abstract out the concept of suit

Othe criteria

Don't include until working model has been developed

* Betting - non-trivial - include later on.
* Side-games
* Card-counting
* Historical/simulation statistics can be coded on demand.

In [360]:
# Import modules

import random
import timeit
from collections import Counter

In [388]:
# Define a dictionary that acts as an unordered card-shoe
# Dictionary keys represent the card as a string, without suit considerations
# Dictionary attributes are a list [no. of cards remaining in deck, value]
# For key 'A', representing an ace, it can have a value of both 1 and 11

def getcards():

    deck = {}
    deck['A'] = [4, 1, 11]

    for card in range(2, 11):
        deck[str(card)] = [4, card]
    
    for card in "JQK":
        deck[str(card)] = [4, 10]
        
    return deck

In [389]:
deck = getcards()
print(deck)

{'A': [4, 1, 11], '2': [4, 2], '3': [4, 3], '4': [4, 4], '5': [4, 5], '6': [4, 6], '7': [4, 7], '8': [4, 8], '9': [4, 9], '10': [4, 10], 'J': [4, 10], 'Q': [4, 10], 'K': [4, 10]}


In [390]:
# Approximately simulate shuffling deck of ordered cards

def shuffler():
    shuffled_shoe = []
    while len(shuffled_shoe) < 52:
        draw = random.choice(list(deck))
        if deck[draw][0] == 0:
            pass
        else:
            deck[draw][0] -= 1
            shuffled_shoe.append(draw)
    return shuffled_shoe

In [391]:
shuffled_shoe = shuffler()
print(shuffled_shoe)

['3', '6', '7', '4', 'J', 'Q', '10', 'A', '6', '5', '9', '10', 'Q', '6', 'K', 'A', '5', '3', 'Q', '5', '4', '9', '6', 'K', 'K', '7', '8', 'A', 'J', '7', '2', '3', '8', '3', '8', '9', '5', '9', 'J', '10', '7', '4', 'K', '8', 'J', 'Q', '4', '10', 'A', '2', '2', '2']


In [365]:
len(shuffled_shoe)

104

In [366]:
# Sanity check on number of cards in shuffled shoe, and on number of each card

if len(shuffled_shoe) == 52:
    print("Yes there are currently 52 cards in the now freshly shuffled shoe.")
else:
    print("Debug...")
    
if any(tally < 4 for tally in list(Counter(shuffled_shoe).values())):
    print("The integrity of the casino's shuffling processes has been compromised.")
else:
    print("And there are the correct number of each card in the deck.")

Debug...
And there are the correct number of each card in the deck.


In [392]:
# Simulate dealing of opening hands module. 
# If the player has a Blackjack, then that player is declared to have won instantly. The house pays out to that player at 3:2.

def openinghand():
    player = []
    dealer = []

    player.append(shuffled_shoe.pop())
    player.append(shuffled_shoe.pop())
    
    dealer.append(shuffled_shoe.pop())
    
    if ('A' in player) and (('10' in player) or ('J' in player) or ('Q' in player) or ('K' in player)):
        print("Blackjack, house pays out at 3:2")
    else:
        print("No Blackjack this time")
        
    return player, dealer

In [393]:
player, dealer = openinghand()

No Blackjack this time


In [394]:
player, dealer

(['2', '2'], ['2'])

In [395]:
# Simulate the player's decision on whether or not to draw cards
# For now, use a hit if under 18 rule of thumb
# O/S Look up decision from a basic strategy table - now complete
# O/S Coding soft vs hard hands - now complete

def playerturn():
    
    """Evaluates the player's score, and keeps drawing cards until the player has a score of more than 18"""
    
    cardvalues = []
    for card in player:
        cardvalues.append(deck[card][1])

    while sum(cardvalues) < 18:
        drawcard = shuffled_shoe.pop()
        player.append(drawcard)
        cardvalues.append(deck[drawcard][1])
        print("Decided to hit, as I'm a risk lover and under 18")
        
    if sum(cardvalues) > 21:
        print("...I've gone bust. Why is my lucky charm that I got from the Amazonian shaman not working?")
    else:
        print("Decided to stick")
        
    print("Player Score:" + str(sum(cardvalues)))
    print(player)
        
    return player, sum(cardvalues)

In [396]:
player, playerscore = playerturn()

Decided to hit, as I'm a risk lover and under 18
Decided to hit, as I'm a risk lover and under 18
Decided to hit, as I'm a risk lover and under 18
Decided to stick
Player Score:19
['2', '2', 'A', '10', '4']


In [397]:
# Simulate the dealer's position, which is fairly clear-cut - have to specify the must-draw on 16 and stick on 17
# O/S - coding soft vs hard hands

def dealerturn():
    
    """Evaluates the dealer's score, and keeps drawing cards until dealer has a score of 17 or more."""
    
    cardvalues = []
    for card in dealer:
        cardvalues.append(deck[card][1])
        
    while sum(cardvalues) <= 16:
        drawcard = shuffled_shoe.pop()
        dealer.append(drawcard)
        cardvalues.append(deck[drawcard][1])
        print("Dealer draws a card as under or equal to 16")
    
    if sum(cardvalues) > 21:
        print("Dealer goes bust")
    else:
        print("Dealer sticks on 17 or over")
    
    print("Dealer Score:" + str(sum(cardvalues)))
    print(dealer)
    
    return dealer, sum(cardvalues)

In [398]:
dealer, dealerscore = dealerturn()

Dealer draws a card as under or equal to 16
Dealer draws a card as under or equal to 16
Dealer goes bust
Dealer Score:22
['2', 'Q', 'J']


In [399]:
# Evaluate the outcome of the hand. The dealer's score is evaluated first, and that is used to evaluate against the score of
# the remaining players' hands, forming the basis for specifying additional players.

def handoutcome(playerscore, dealerscore):

    if playerscore > 21:
        if dealerscore > 21:
            print("Player busts. Dealer also busts. But the 'edge' means that player loses the hand.")
        else:
            print("Player busts. Dealer wins the hand.")
    else:
        if dealerscore > 21:
            print("Dealer busts. Player wins the hand")
        else:
            if dealerscore > playerscore:
                print("Dealer has higher score. Dealer wins the hand")
            elif dealerscore < playerscore:
                print("Player has higher score. Player wins the hand")
            else:
                print("Both dealer and player have the same score, the outcome of the hand is a draw")

In [400]:
handoutcome(playerscore, dealerscore)

Dealer busts. Player wins the hand


Objective 1a - build a working model of Blackjack - complete

Have now a bare skeleton working module that can simulate one round of blackjack, with some O/S caveats as indicated above.